## 2. Musical notes and harmonics, chords, consonances and dissonances




In [6]:
import numpy as np
import matplotlib.pyplot as plt
import scipy.signal as ss
from matplotlib.ticker import LogFormatter
import simpleaudio as sa
from ipywidgets import interact
import ipywidgets as widgets

In [7]:
fs = 44100  # 44100 samples per second
LA = 440    # LA4
fnote = [LA/2 * 2**(i/12) for i in range(3, 15)]
fnote

[261.6255653005986,
 277.1826309768721,
 293.6647679174076,
 311.1269837220809,
 329.6275569128699,
 349.2282314330039,
 369.9944227116344,
 391.99543598174927,
 415.3046975799451,
 440.0,
 466.1637615180899,
 493.8833012561241]

In [8]:
class Chord:
    def __init__(self, tonic, duration, harmonics):
        self.tonic = tonic
        self.duration = duration
        self.harmonics = harmonics

    def _notetofr(self, i):
        return fnote[i % 12] * (1 + int(i / 12)) * (0.5 if i < 0 else 1)
    
    def _harmtoampl(self, h):
        return 1 / (h+1) ** (4 if h % 2 > 0 else 2)

    def note(self, f):
        '''Generate a np array for a given note index or frequency (duration is an external constant),
        # where n = 0 means DO3. If f is a float, then it is directly used as a frequency'''
        # generate array with duration*sample_rate steps, ranging between 0 and duration
        t = np.linspace(0, self.duration, int(self.duration * fs), False)
        # get the note's frequency
        if isinstance(f, int):
            f = self._notetofr(f)

        # generate the fundamental wave
        w = np.sin(2 * np.pi * f * t)
        # add the harmonics with given weights
        for h in range(self.harmonics):
            w += np.sin(2 * np.pi * f * (h+2) * t) * self._harmtoampl(h+1)

        return w

    def dichord(self, interval):
        return self.note(self.tonic) + self.note(self.tonic + interval)
    
    def Tartini(self, interval):
        return self.note(self._notetofr(self.tonic + interval) - self._notetofr(self.tonic))
    
    def _3chord(self, int1, int2):
        return self.note(self.tonic) + self.note(self.tonic + int1) + self.note(self.tonic + int1 + int2)

    def maj(self):
        return self._3chord(4, 3)

    def min(self):
        return self._3chord(3, 4)

    def sus(self):
        return self._3chord(5, 2)

    def Vaug(self):
        return self._3chord(4, 4)

    def Vdim(self):
        return self._3chord(3, 3)

    def add7(self):
        return self.note(self.tonic + 4+3+3)

    def add7maj(self):
        return self.note(self.tonic + 4+3+4)

    def VII(self):
        return self.maj() + self.add7()
    
    def VIImaj(self):
        return self.maj() + self.add7maj()

    def minVII(self):
        return self.min() + self.add7()

    def minVIImaj(self):
        return self.min() + self.add7maj()

    def add9(self):
        return self.note(self.tonic + 4+3+3+4)

    def IX(self):
        return self.VII() + self.add9()

    def IXmaj(self):
        return self.VIImaj() + self.add9()


In [9]:
def play(audio):
    # ensure that highest value is in 16-bit range
    playable = audio * (2**15 - 1) / np.max(np.abs(audio))
    # stop any ongoing play
    #sa.stop_all()
    # convert to 16-bit data and play
    return sa.play_buffer(playable.astype(np.int16), 1, 2, fs)

In [10]:
def plottimefreq(s, duration):
    # plot the time series
    plt.subplots(figsize=(15, 5)) 
    ax = plt.subplot(1, 2, 1)
    plt.plot(np.real(s))
    plt.xlim(0)
    ax.set_xlabel('t [ms]')
    maxx = int(duration*10 + 1)*100
    ax.set_xticks(np.arange(0, maxx*fs/1000, maxx*fs/10000, dtype=int))
    ax.set_xticklabels(np.arange(0, maxx, maxx/10, dtype=int))
    plt.grid(which='major')
    plt.title("Wave packet")

    # also compute and plot power spectrum
    ax = plt.subplot(1, 2, 2)
    s = np.pad(s, (0, 65536-s.size), mode='constant')
    W = np.abs(np.fft.fft(s) ** 2)
    f = np.fft.fftfreq(s.size, 1/(2*fs))
    plt.plot(f, W)
    plt.xlim(20, 15000)
    #formatter = LogFormatter(labelOnlyBase=False, minor_thresholds=(1, 0.1))
    #ax.get_xaxis().set_minor_formatter(formatter)
    ax.set_xlabel('f [Hz]')
    plt.ylim(1E-4)
    plt.xscale('log')
    plt.yscale('log')
    plt.grid(which='both')
    plt.title("Power spectrum (log/log)")
    plt.show()


In [11]:
@interact(tonic=widgets.IntSlider(min=-11, max=11, value=0, continuous_update=False),
          interval=widgets.IntSlider(min=0, max=12, value=0, continuous_update=False),
          Tartini=widgets.RadioButtons(options=['no', 'only', 'add']),
          duration=widgets.FloatSlider(min=0.01, max=1.48, value=0.6, step=0.01, continuous_update=False),
          harmonics=widgets.IntSlider(min=0, max=5, value=0, continuous_update=False),
          window=widgets.RadioButtons(options=['hanning', 'rect', 'gaussian'])
         )
def interactiveplay(tonic, interval, Tartini, duration, harmonics, window):
    # generate signal for the given chord
    c = Chord(tonic, duration, harmonics)
    if interval > 0:
        if Tartini != 'no':
            t = c.Tartini(interval)
        if Tartini == 'add':
            s = c.dichord(interval) + t
        elif Tartini == 'only':
            s = t
        else:
            s = c.dichord(interval) 
    else:
        s = c.dichord(interval)
    # use a window to smooth begin and end
    if window == 'hanning':
        s *= np.hanning(s.size)
    elif window == 'gaussian':
        s *= ss.gaussian(s.size, 5000*duration)
    plottimefreq(s, duration)
    play(s)
    # wait for playback to finish before exiting
    #p.wait_done()

interactive(children=(IntSlider(value=0, continuous_update=False, description='tonic', max=11, min=-11), IntSl…

In [12]:
tonic = 0
h = 1
duration = 1.5

@interact(tonic=widgets.IntSlider(min=0, max=11, value=0, continuous_update=False),
          chord=widgets.RadioButtons(options=['maj', 'min', 'sus', 'Vaug', 'Vdim', 'VII', 'VIImaj', 'minVII', 'minVIImaj', 'IX', 'IXmaj']),
          duration=widgets.FloatSlider(min=0.01, max=1.48, value=0.6, step=0.01, continuous_update=False),
          harmonics=widgets.IntSlider(min=0, max=5, value=0, continuous_update=False))
def interactiveplay(tonic, chord, duration, harmonics):
    c = Chord(tonic, duration, harmonics)
    s = getattr(c, chord)()
    s *= np.hanning(s.size)
    plottimefreq(s, duration)
    play(s)

interactive(children=(IntSlider(value=0, continuous_update=False, description='tonic', max=11), RadioButtons(d…

### Uncertainty: practical case

We can assess the psico-acustic effects of the uncertainty by correlating the *critical band* with the shortest duration of a sound for it to be perceived with a defined pitch. The critical band for a frequency $f$ can be approximated with [ https://en.wikipedia.org/wiki/Critical_band ]:

$$CB = 24.7 \cdot (4.37 f / 1000 + 1)$$

For example, for a sound of 100 Hz, the critical band is about 35 Hz. Therefore, a 100 Hz sound must have a bandwidth $2\sigma_f \le$ 35 Hz in order to have a clearly distinguishable pitch (e.g. from another sound). The uncertainty suggests that the pitch nevertheless becomes undefined if the sound duration $2\sigma_t \le 2 / (4\pi \cdot \sigma_f) \simeq$ 90 ms. Higher frequency sounds have a larger critical band, therefore the duration limit would be even shorter. In practice, our hears have intrinsic limitations that enters into play at about the same levels of resolution if not lower.
